In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('/Users/vancescadinh/Documents/house-price-prediction/01-data-processing/data/data_numerical.parquet', engine='pyarrow')
df = df.drop(columns=['address', 'proceedings for breach of planning regulations'])
df.head(2)

,price,construction year,building condition,asbestos certificate,living area,bedrooms,bathrooms,toilets,primary energy consumption,energy class,...,planning permission obtained,subdivision permit,possible priority purchase right,non-flood zone,g-score,shared building,surface of the plot,sewer network connection,designated land use,double glazing
uuid,,,,,,,,,,,,,,,,,,,,,
bff0933c-8706-450e-be72-df1c836eb396,765000.0,2015.0,1.0,0.0,171.0,2.0,1.0,2.0,102.0,4.0,...,1,0.0,1.0,1,1.0,0.0,NaN,NaN,NaN,NaN
48103edf-d945-4052-a349-31a1bdee8300,321477.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.impute import KNNImputer

In [5]:
pipeline = make_pipeline(
    StandardScaler(),
    KNNImputer(n_neighbors=3)
)

df_imputed = pd.DataFrame(pipeline.fit_transform(df), columns=df.columns)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

In [10]:
X = df_imputed.drop(columns='price')
y = df_imputed['price']

# Standardize
# X_normalized = StandardScaler().fit_transform(X)
# Select best features
kbest = SelectKBest(score_func=f_regression, k=12)
X_kbest = kbest.fit_transform(X, y)
X.columns[kbest.get_support()]

Index(['building condition', 'asbestos certificate', 'living area', 'bedrooms',
       'bathrooms', 'toilets', 'energy class', 'subdivision permit',
       'non-flood zone', 'g-score', 'surface of the plot',
       'designated land use'],
      dtype='object')

In [11]:
kbest_df = pd.DataFrame(X_kbest, columns=X.columns[kbest.get_support()])
kbest_df.head(2)

,building condition,asbestos certificate,living area,bedrooms,bathrooms,toilets,energy class,subdivision permit,non-flood zone,g-score,surface of the plot,designated land use
0,-1.704509,-1.095445,-0.180256,-0.910142,-0.34181,0.199653,-0.795582,-0.602303,0.692526,-0.524604,-0.205709,-0.471694
1,0.954990,0.243432,-0.100656,0.310010,-0.34181,0.582772,1.518519,1.660295,-1.443990,-0.179833,-0.046877,-0.299748


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    kbest_df, y,
    test_size=0.2,         # 20% for validation
    random_state=2025,       
)

In [13]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

RMSE: 0.76
MAE: 0.49
R²: 0.45


In [14]:
from sklearn.linear_model import ElasticNet

In [29]:
# Initialize ElasticNet model
model = ElasticNet(alpha=.001, l1_ratio=.001, random_state=42)

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

RMSE: 0.76
MAE: 0.49
R²: 0.45
